In [12]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import defaultdict
import pandas as pd

In [2]:
df = pd.read_csv("tweets/train.csv")
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [3]:
imp_features = ["text"]
TARGET = ["sentiment"]

In [5]:
df.groupby('sentiment').describe()

textID                           text         \
           count unique         top freq  count unique   
sentiment                                                
negative    7781   7781  549e992a42    1   7781   7781   
neutral    11118  11118  cb774db0d1    1  11117  11117   
positive    8582   8582  6e0c6d75b1    1   8582   8582   

                                                                   \
                                                         top freq   
sentiment                                                           
negative       Sooo SAD I will miss you here in San Diego!!!    1   
neutral                  I`d have responded, if I were going    1   
positive   2am feedings for the baby are fun when he is a...    1   

          selected_text                    
                  count unique   top freq  
sentiment                                  
negative           7781   5861  miss  143  
neutral           11117  11111    aw    2  
positive           8582   5537  good  198

In [13]:
class NaiveBayesClassifier:
    def __init__(self):
        self.prior = defaultdict(int)
        self.word_counts = defaultdict(lambda: defaultdict(int))
        self.vocab = set()
        self.classes = set()

    def train(self, documents):
        for doc, label in documents:
            self.classes.add(label)
            self.prior[label] += 1
            tokens = word_tokenize(doc.lower())
            for token in tokens:
                if token not in stopwords.words('english') and token.isalpha():
                    self.word_counts[label][token] += 1
                    self.vocab.add(token)

    def classify(self, document):
        max_score = float('-inf')
        argmax_class = None
        tokens = word_tokenize(document.lower())
        for label in self.classes:
            score = 0
            for token in tokens:
                if token in self.vocab:
                    score += self.word_counts[label][token]
            score += self.prior[label]
            if score > max_score:
                max_score = score
                argmax_class = label
        return argmax_class


In [23]:
docs = pd.read_csv("tweets/train.csv")

# Drop rows with missing values
docs = docs.dropna(subset=["text", "sentiment"])

# Convert non-string values to strings in the "text" column
docs['text'] = docs['text'].apply(lambda x: str(x) if isinstance(x, (int, float)) else x)


x =  df["text"]
y =  df["sentiment"]
documents = list(zip(x, y))

In [24]:
classifier = NaiveBayesClassifier()
classifier.train(documents)


AttributeError: 'float' object has no attribute 'lower'

In [ ]:
test_doc = "This movie was great!"
predicted_class = classifier.classify(test_doc)
print(f"Predicted sentiment: {predicted_class}")

In [25]:
# Load your data
docs = pd.read_csv("tweets/train.csv")

# Convert non-string values to strings in the "text" column
docs['text'] = docs['text'].apply(lambda x: str(x) if isinstance(x, (int, float)) else x)

# Drop rows with missing values
docs = docs.dropna(subset=["text", "sentiment"])

x = docs["text"]
y = docs["sentiment"]

# Prepare data for training
documents = []
for text, sentiment in zip(x, y):
    if isinstance(text, str):
        documents.append((text, sentiment))

# Train the Naive Bayes classifier
classifier = NaiveBayesClassifier()
classifier.train(documents)

# Example usage:
test_doc = "This movie was great!"
predicted_class = classifier.classify(test_doc)
print(f"Predicted sentiment: {predicted_class}")


Predicted sentiment: neutral


In [28]:
import dill

# Train the Naive Bayes classifier
classifier = NaiveBayesClassifier()
classifier.train(documents)

# Save the trained model to a file
with open('naive_bayes_model.pkl', 'wb') as f:
    dill.dump(classifier, f)

# Load the saved model from file
with open('naive_bayes_model.pkl', 'rb') as f:
    classifier = dill.load(f)

# Now you can use the classifier for predictions
test_doc = 'Sooo SAD I will miss you here in San Diego!!!'
predicted_class = classifier.classify(test_doc)
print(f"Predicted sentiment: {predicted_class}")


Predicted sentiment: neutral


In [31]:
# Now you can use the classifier for predictions
test_doc = 'bullying me'
predicted_class = classifier.classify(test_doc)
print(f"Predicted sentiment: {predicted_class}")


Predicted sentiment: neutral
